# Pivot from Boavizta Data

### Setup

Workbook which makes a pivot from boavizta's database to use in collaboration with In Kind Direct.

In [1]:
import pandas as pd
import numpy as np

Reads the Boavizta database

In [2]:
df = pd.read_csv('CSVs/boavizta-data/boavizta-data-us.csv')

Get rid of things we won't need. Irrelevant columns and servers.

In [3]:
df = df.drop(columns = ['gwp_error_ratio','gwp_daughterboard_ratio', 'gwp_enclosure_ratio', 'gwp_mainboard_ratio', 'gwp_ssd_ratio', 'height','number_cpu'])
df = df.loc[df["category"] == 'Workplace' ]

## Sampling the Data

Take a look at the CSV.<br>
Take a sample and see what the columns are.

In [4]:
df.sample(5)

,manufacturer,name,category,subcategory,gwp_total,gwp_use_ratio,yearly_tec,lifetime,use_location,report_date,...,assembly_location,screen_size,server_type,hard_drive,memory,added_date,add_method,gwp_transport_ratio,gwp_eol_ratio,comment
1075,Lenovo,V410z,Workplace,Desktop,875.0,0.440,NaN,NaN,WW,May 2017,...,CN,21.0,NaN,NaN,NaN,01-11-2020,Initial Parsing,NaN,NaN,NaN
1068,Lenovo,V110-17ISK,Workplace,Laptop,408.0,0.440,NaN,5.0,US,November 2016,...,CN,17.3,NaN,NaN,NaN,01-11-2020,Initial Parsing,NaN,NaN,NaN
475,Dell,Precision Tower 7820,Workplace,Desktop,1820.0,0.688,597.21,4.0,EU,December 2018,...,EU,NaN,NaN,NaN,NaN,01-11-2020,Initial Parsing,NaN,NaN,NaN
1125,Microsoft,Surface Go 2,Workplace,Tablet,107.0,0.252,12.02,3.0,NaN,October 2021,...,NaN,NaN,NaN,NaN,NaN,2022-09-15,Microsoft Auto Parser,0.028,0.009,NaN
704,HP,Pro x360 Fortis 11 inch G9 Notebook PC,Workplace,Laptop,325.0,0.080,12.63,4.0,North America,2022-2-6,...,China,11.0,NaN,NaN,NaN,2022-04-09,HP Auto Parser,0.030,0.000,NaN


Take a look at the columns that are there, with an explanation from boavizta below the cell.

In [5]:
df.columns.to_list

<bound method IndexOpsMixin.tolist of Index(['manufacturer', 'name', 'category', 'subcategory', 'gwp_total',
       'gwp_use_ratio', 'yearly_tec', 'lifetime', 'use_location',
       'report_date', 'sources', 'sources_hash', 'gwp_manufacturing_ratio',
       'weight', 'assembly_location', 'screen_size', 'server_type',
       'hard_drive', 'memory', 'added_date', 'add_method',
       'gwp_transport_ratio', 'gwp_eol_ratio', 'comment'],
      dtype='object')>

- <b>manufacturer</b>: Manufacturer name, e.g. "Dell" or "HP"<br>
- <b>name</b>: Product name
- <b>category</b>:
    - <b>Workplace</b>: product commonly used in a workplace
    - <b>Datacenter</b>: product commonly used in a data center (e.g. server, network switch, etc.)
- <b>gwp_total</b>: GHG emissions (estimated as CO2 equivalent, the unit is kgCO2eq) through the total lifecycle of the product (Manufacturing, Transportation, Use phase and Recycling)
- <b>gwp_use_ratio</b>: part of the GHG emissions coming from the use phase (the hypothesis for this use phase are detailed in the other columns, especially the lifetime and the use_location)
- <b>yearly_tec</b>: Yearly estimated energy demand in kWh
- <b>lifetime</b>: Expected lifetime (in years)
- <b>use_location</b>: The region of the world in which the device usage footprint has been estimated.
    - <b>US:</b> United States of America
    - <b>EU:</b> Europe
    - <b>DE:</b> Germany
    - <b>CN:</b> China
    - <b>WW:</b> Worldwide
- <b>report_date</b>: the date at which the Product Carbon Footprint report of the device was published
- <b>sources</b>: the original URLs from which the data for this row was sourced
- <b>gwp_error_ratio</b>: the datasheets commonly come with a diagram that shows the error margin for the footprint gwp_manufacturing_ratio part of the GHG emissions coming from the manufacturing phase
- <b>weight</b>: product weight in kg
- <b>assembly_location</b>: The region of the world in which the device is assembled
    - <b>US:</b> United States of America
    - <b>EU:</b> Europe
    - <b>CN:</b> China
    - <b>Asia</b>: Asia
- <b>screen_size</b>: in inches
- <b>server_type</b>: the type of server
- <b>hard_drive</b>: the hard drive of the device if any
- <b>memory</b>: RAM in GB
- <b>number_cpu</b>: number of CPUs
- <b>height</b>: the height of the device in a datacenter rack, in U
- <b>added_date</b>: the date at which this row was added
- <b>add_method</b>: how was the data for this row collected

## Transforming the Data

We want to average the collection of data over two categories:
- Subcategory (e.g. Laptop)
- A screen upper and lower bound (e.g. greater than or equal to 14")<br>

The upper and lower bound will be different for each subcategory.<br>
For some subcategories, say printer, there won't be a relevant screen bound.<br><br>
This will involve <b>splitting</b> the data, <b>applying</b> some transformations and then <b>rejoining</b>.<br><br> I'll save you most of the details. I'd recommend taking a look at the code just if you want to verify it.<br><br>
I use the same function extensively to make the screen upper and lower bounds, which I define below.<br>
I also make a group by object, which groups by subcategory.

In [6]:
def group_screens(screen_size, number):
    if screen_size < number:
        return "<"+str(number)
    elif screen_size >= number:
        return str(number)+"+"
    else:
        return 0

subcats = df.copy().groupby('subcategory')

### Tranforming Monitors

In [7]:
#Makes a copy the part of the grouped dataframe which where subcategory is monitor
monitor = subcats.get_group('Monitor').copy()
#Applies screen group, with an upper and lower bound of 33
monitor['screen_group'] = monitor.apply(lambda x: group_screens(x['screen_size'],
                                                                number=33),
                                                                axis=1)
#Gets an average of *all* of the monitors. This is useful for when our data lacks screen size.
avg_monitor = monitor.mean()
avg_monitor['subcategory'] = 'Average Monitor'
#Drops the monitors (already used in the average) which lack a recorded screen size
monitor = monitor[monitor['screen_size'].notna()]
monitor['subcategory'] = monitor['subcategory'] + " " + monitor['screen_group']

C:\Users\TomBurdge\AppData\Local\Temp\ipykernel_22780\2784464728.py:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  avg_monitor = monitor.mean()


### Rinse and repeat this transformation for Laptops

In [8]:
#Makes a copy the part of the grouped dataframe which where subcategory is laptop
laptop = subcats.get_group('Laptop').copy()
#Applies screen group, with an upper and lower bound of 14
laptop['screen_group'] = laptop.apply(lambda x: group_screens(x['screen_size'],
                                                              number=14), 
                                                              axis=1)
#Gets an average of all of the laptops.
avg_laptop = laptop.mean()
avg_laptop['subcategory'] = 'Average Laptop'
#Drops the monitors (already used in the average) which lack a recorded screen size
laptop = laptop[laptop['screen_size'].notna()]
laptop['subcategory'] = laptop['subcategory'] + " " + laptop['screen_group']

C:\Users\TomBurdge\AppData\Local\Temp\ipykernel_22780\695722128.py:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  avg_laptop = laptop.mean()


### Need to do something slightly different for desktops that don't have a screen size.

For desktops, it is standard to distinguish between desktops (without screends) and desktop all-in-ones (with screen, keyboard, and mouse).<br>
Boavizta's database doesn't do this, but <b>we<b/> can by distinguishing between whether it has a screen or not

In [9]:
#Makes a copy the part of the grouped dataframe which where subcategory is desktop
boavizta_desktop = subcats.get_group('Desktop').copy()
#Applies screen group, with an upper and lower bound of 14
boavizta_desktop['screen_group'] = boavizta_desktop.apply(lambda x: 
                                                 group_screens(x['screen_size'],
                                                 number=24), 
                                                 axis=1)
#Drops the monitors (already used in the average) which lack a recorded screen size
all_in_one = boavizta_desktop[boavizta_desktop['screen_size'].notna()]
#assigns a new subcategory as all in one to this new subgroup along with the screen group
all_in_one['subcategory'] = "All-in-One " + all_in_one["screen_group"]
#Gets an average of all desktops with a screen size. These are called an 'All-in-One'
avg_a_i_o = all_in_one.mean()
avg_a_i_o['subcategory'] = 'Average All-in-One'

C:\Users\TomBurdge\AppData\Local\Temp\ipykernel_22780\2085705383.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_in_one['subcategory'] = "All-in-One " + all_in_one["screen_group"]
C:\Users\TomBurdge\AppData\Local\Temp\ipykernel_22780\2085705383.py:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  avg_a_i_o = all_in_one.mean()


Above, we have treated All-in-Ones, now we need to treat the desktops without a screen and keyboard.
The laptops without a screen are:

In [10]:
desktop = boavizta_desktop[~boavizta_desktop['screen_size'].notna()]

Let's make an average of the desktops.

In [11]:
avg_desktop = desktop.mean()
avg_desktop['subcategory'] = 'Average Desktop'

C:\Users\TomBurdge\AppData\Local\Temp\ipykernel_22780\4123310747.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  avg_desktop = desktop.mean()


Now, we need to define a function that will break them into each of the main sizes:
- Ultra Small Desktop
- Small Form Factor
- Tower

For this, we'll need the regex module to find matching products.

In [12]:
import re

Here we use a simple regex to get the desktops of the various types:

In [13]:
#Lots of desktop towers which contain 'Tower' in the name cell
desktop_tower = desktop[desktop['name'].str.contains(r'Tower')==True]
#make the dataframe's subcategory Desktop Tower
desktop_tower['subcategory'] = 'Desktop Tower'

# Small form factors which contain 'SFF' or 'Small Form Factor' in the name cell
desktop_SFF = desktop[desktop['name'].str.contains(r'SFF')==True]
desktop_SFF = pd.concat([desktop_SFF, 
                                  desktop[desktop['name'].str.contains(r'Small Form Factor')==True]])
#make the dataframe's subcategory Desktop SFF
desktop_SFF['subcategory'] = 'Desktop SFF'

#Ultra Small Desktops which contain the following in the 'name' cell
desktop_USDT = desktop[desktop['name'].str.contains(r'USDT')==True]
desktop_USDT = pd.concat([desktop_USDT,
                        desktop[desktop['name'].str.contains(r'Tiny')==True],
                        desktop[desktop['name'].str.contains(r'Micro')==True],
                        desktop[desktop['name'].str.contains(r'Nano')==True]])
#make the dataframe's subcategory Desktop USDT
desktop_USDT['subcategory'] = 'Desktop USDT'

C:\Users\TomBurdge\AppData\Local\Temp\ipykernel_22780\3208305278.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  desktop_tower['subcategory'] = 'Desktop Tower'


## Now, we can rejoin the entire Dataframe

We have the following dataframes named:
- monitor
- avg_monitor
- laptop
- avg_laptop
- all_in_one
- avg_a_i_o
- desktop_tower
- desktop_SFF
- desktop_USDT
- avg_desktop
<br><br>
<i>(Note: we made separate desktops for the averages because all columns need to be matching. We could have done join but it's no less fiddly)</i>

Now, we can rejoin all of dataframes, where the subcategories are those that we wish to pivot over.

In [14]:
df_list = [monitor, avg_monitor.to_frame().transpose(),
           laptop, avg_laptop.to_frame().transpose(),
           all_in_one, avg_a_i_o.to_frame().transpose(), 
           desktop_tower, desktop_SFF, desktop_USDT, avg_desktop.to_frame().transpose()]
df_rejoined = pd.concat(df_list)

In [15]:
df['subcategory'].value_counts()

Laptop         447
Monitor        294
Desktop        255
Smartphone      40
Tablet          14
Thin Client     11
Printer         11
Thin client      5
Workstation      4
Name: subcategory, dtype: int64

Now, we need to rejoin this with <u>those subcategories which we did not need to modify</u><br>
These were members of the subcategories:
- Tablet
- Smartphone
- Printer
- Workstation
<br><br>
To do this, we can concat them with rejoined:

In [16]:
df_unchanged = df[(df['subcategory']=='Tablet') 
                  |(df['subcategory']=='Smartphone')
                  |(df['subcategory']=='Printer')
                  |(df['subcategory']=='Workstation')]

df_rejoined = pd.concat([df_rejoined, df_unchanged])

We now have the dataframe that we will pivot over.

## Make a pivot from the joined dataframe

All we need to do is pivot the numerical values into a mean across subcategories.

In [17]:
df_pivot = df_rejoined.copy().pivot_table(index = 'subcategory', aggfunc = 'mean')

#df_pivot.to_csv('product-categories.csv')

df_pivot

C:\Users\TomBurdge\AppData\Local\Temp\ipykernel_22780\3960562440.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  df_pivot = df_rejoined.copy().pivot_table(index = 'subcategory', aggfunc = 'mean')


,gwp_eol_ratio,gwp_manufacturing_ratio,gwp_total,gwp_transport_ratio,gwp_use_ratio,lifetime,screen_size,weight,yearly_tec
subcategory,,,,,,,,,
All-in-One 24+,0.009333,0.503706,729.400586,0.093133,0.357034,3.526316,25.810345,9.207789,92.733050
All-in-One <24,0.005573,0.754356,442.702627,0.049024,0.392504,3.972973,8.932203,4.361871,55.025867
Average All-in-One,0.005852,0.718246,499.262088,0.055776,0.385261,3.907692,12.261905,5.043889,57.671985
Average Desktop,NaN,0.447657,673.668815,NaN,0.475491,4.171296,NaN,7.593024,170.416714
Average Laptop,0.002377,0.798919,309.873826,0.034445,0.189894,4.135731,13.927139,1.823987,20.246444
Average Monitor,0.004243,0.618966,566.142007,0.040441,0.333311,5.404762,25.356794,7.111847,55.863532
Desktop SFF,NaN,0.472667,624.448276,NaN,0.485931,4.000000,NaN,5.619091,97.027222
Desktop Tower,NaN,0.387632,869.588235,NaN,0.537441,4.205882,NaN,11.012143,252.133158
Desktop USDT,NaN,0.605500,357.582476,NaN,0.316000,4.095238,NaN,2.494533,40.046667


## Get Device Proportional Amounts

For this calculation, we need to get the actual average **amounts** as a percent of total for:
- New manufacturing
- Transport from new production and to waste handling
- End-of-life waste handling

Currently, we just have the ratio not the value. We can get this quite easily:<br>
(We need to make the NaNs zeros for the next step.)

In [18]:
df_pivot['gwp_manufacturing_val']=df_pivot['gwp_total']*df_pivot['gwp_manufacturing_ratio'].fillna(0)
df_pivot['gwp_transport_val']=df_pivot['gwp_total']*df_pivot['gwp_transport_ratio'].fillna(0)
df_pivot['gwp_eol_val']=df_pivot['gwp_total']*df_pivot['gwp_eol_ratio'].fillna(0)

From this, we can know the figure that will go in our calculation (the sum of the three above columns we just made):

In [19]:
#Now, actually making the column
df_pivot['gwp_man_trans_eol_sum'] = df_pivot['gwp_manufacturing_val']+df_pivot['gwp_transport_val']+df_pivot['gwp_eol_val']

## Perform miscellaneous data cleaning

There's no data for the ratios for printers in the boavizta file for printers.<br>Having looked at the source files for the boavizta data, the total is so high because they factor in the use emissions as very high from using paper.<br>Roughly, the non-use amount is 5% for printers. So, we'll **assume** a very low figure of 5% for manufacturing, transport, and eol as a % of total:

In [20]:
df_pivot.loc['Printer','gwp_man_trans_eol_sum'] = df_pivot.loc['Printer','gwp_total']*0.05

There's also no data for workstation and printer weight.<br>
For a printer, we'll assume a weight of 26kg and for workstation a weight of 15kg.

In [21]:
df_pivot.loc['Printer','weight'] = 26
df_pivot.loc['Workstation','weight'] = 15

# Saving

I've saved this in the CSVs file as 'boavizta-product-averages.csv'

In [22]:
#df_pivot.to_csv('CSVs/boavizta-product-averages.csv')